# Customer Orders - Delta Live Tables
This Notebook demonstrates how to create a notebook for a Delta Live Table Pipeline. 

If you have any issues starting your cluster please modify the pipeline settings via JSON and overwrite the cluster config with the snippet below

    "clusters": [
        {
            "label": "default",
            "node_type_id": "Standard_DS3_v2",
            "driver_node_type_id": "Standard_DS3_v2"
        }
    ],

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Please update the filepath accordingly
orders_path = "/FileStore/tables/RawData/orders_full.csv"

orders_schema = StructType([
                    StructField("ORDER_ID", IntegerType(), False),
                    StructField("ORDER_DATETIME", StringType(), False),
                    StructField("CUSTOMER_ID", IntegerType(), False),
                    StructField("ORDER_STATUS", StringType(), False),
                    StructField("STORE_ID", IntegerType(), False)
                    ]
                    )

In [0]:
@dlt.table
def orders_rawdata():
    df = spark.read.format("csv").option("header",True).schema(orders_schema).load(orders_path)
    return df

orders_rawdata is defined as a
 Delta Live Tables dataset
 with schema: 
 

 
 
 
 
 
 Name 
 Type 
 
 
 
 ORDER_ID 
 int 
 

 
 ORDER_DATETIME 
 string 
 

 
 CUSTOMER_ID 
 int 
 

 
 ORDER_STATUS 
 string 
 

 
 STORE_ID 
 int 
 
 
 
 

 
 To populate your table you must either:
 
 
 Run an existing pipeline using the
 Delta Live Tables menu
 
 
 Create a new pipeline: Create Pipeline

In [0]:
# Please update the filepath accordingly
order_items_path = "/FileStore/tables/RawData/order_items_full.csv"

order_items_schema = StructType([
                    StructField("ORDER_ID", IntegerType(), False),
                    StructField("LINE_ITEM_ID", IntegerType(), False),
                    StructField("PRODUCT_ID", IntegerType(), False),
                    StructField("UNIT_PRICE", DoubleType(), False),
                    StructField("QUANTITY", IntegerType(), False)
                    ]
                    )

In [0]:
@dlt.table
def order_items_rawdata():
    return spark.read.format("csv").option("header",True).schema(order_items_schema).load(order_items_path)

order_items_rawdata is defined as a
 Delta Live Tables dataset
 with schema: 
 

 
 
 
 
 
 Name 
 Type 
 
 
 
 ORDER_ID 
 int 
 

 
 LINE_ITEM_ID 
 int 
 

 
 PRODUCT_ID 
 int 
 

 
 UNIT_PRICE 
 double 
 

 
 QUANTITY 
 int 
 
 
 
 

 
 To populate your table you must either:
 
 
 Run an existing pipeline using the
 Delta Live Tables menu
 
 
 Create a new pipeline: Create Pipeline

In [0]:
@dlt.table
def orders_processed():
    return (
        dlt.read("orders_rawdata")
              .select(
              'ORDER_ID', \
              to_date('ORDER_DATETIME', "dd-MMM-yy kk.mm.ss.SS").alias('ORDER_DATE'), \
              'CUSTOMER_ID', \
              'ORDER_STATUS', \
              'STORE_ID',
              current_timestamp().alias("MODIFIED_DATE")
               )
          )

orders_processed is defined as a
 Delta Live Tables dataset
 with schema: 
 

 
 
 
 
 
 Name 
 Type 
 
 
 
 ORDER_ID 
 int 
 

 
 ORDER_DATE 
 date 
 

 
 CUSTOMER_ID 
 int 
 

 
 ORDER_STATUS 
 string 
 

 
 STORE_ID 
 int 
 

 
 MODIFIED_DATE 
 timestamp 
 
 
 
 

 
 To populate your table you must either:
 
 
 Run an existing pipeline using the
 Delta Live Tables menu
 
 
 Create a new pipeline: Create Pipeline

In [0]:
@dlt.table
def order_items_processed():
    return (
        dlt.read("order_items_rawdata")
              .select(
              'ORDER_ID', \
              'PRODUCT_ID', \
              'UNIT_PRICE', \
              'QUANTITY',
              current_timestamp().alias("MODIFIED_DATE")
               )
          )

order_items_processed is defined as a
 Delta Live Tables dataset
 with schema: 
 

 
 
 
 
 
 Name 
 Type 
 
 
 
 ORDER_ID 
 int 
 

 
 PRODUCT_ID 
 int 
 

 
 UNIT_PRICE 
 double 
 

 
 QUANTITY 
 int 
 

 
 MODIFIED_DATE 
 timestamp 
 
 
 
 

 
 To populate your table you must either:
 
 
 Run an existing pipeline using the
 Delta Live Tables menu
 
 
 Create a new pipeline: Create Pipeline

In [0]:
# We can create pipeline now from Workflows -> Delta Live Tables
# Give product edition as core
# Pipeline mode Triggered
# Leave cluster policy as None
# In Source Code navigate to Notebook folder and select this notebook
# Storage location /FileStore/tables/dlt_schema/ and Target Schema as dlt_schema
#  Cluster mode - fixed Workers - 1
